# 线性回归的简洁实现

通过使用深度学习框架来简洁地实现
线性回归模型
生成数据集

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [5]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 0.7882, -0.7068],
         [ 0.5081,  0.2577],
         [-0.5769,  0.1545],
         [-0.3271, -0.6080],
         [-0.2716, -1.4628],
         [-1.1530, -1.4643],
         [ 0.1635, -0.2018],
         [-0.0753, -1.1161],
         [ 3.4251,  0.1953],
         [ 0.3589, -0.9478]]),
 tensor([[ 8.1742],
         [ 4.3357],
         [ 2.5157],
         [ 5.6106],
         [ 8.6395],
         [ 6.8726],
         [ 5.2155],
         [ 7.8377],
         [10.3918],
         [ 8.1590]])]

使用框架的预定义好的层

In [6]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数

In [8]:
loss = nn.MSELoss()

实例化一个`SGD`实例

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练过程代码与我们从零开始实现时所做的非常相似

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000157
epoch 2, loss 0.000094
epoch 3, loss 0.000094


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [11]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0008,  0.0006])
b的估计误差： tensor([0.0013])
